### Data Extraction

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import zipfile
import os

In [ ]:
Zip = "baseballdatabank-2022.2.zip"
Zip_Path = Zip.replace(".zip", "")
Odp = "data"
Dp = f"{Odp}/{Zip_Path}"
Core = f"{Dp}/core"
Contrib = f"{Dp}/contrib"
if Odp not in os.listdir() or Zip_Path not in os.listdir(Odp):
    print("Extracting Archive")
    with zipfile.ZipFile(Zip, "r") as data_archive:
        data_archive.extractall(Odp)
else:
    print("Archive already exists")

### Dataframe Creation from required Dataset's

In [ ]:
dfsal = pd.read_csv(f'{Contrib}/Salaries.csv')
dfplayers = pd.read_csv(f'{Core}/People.csv')
dfschool = pd.read_csv(f'{Contrib}/Schools.csv')
dfcollege = pd.read_csv(f'{Contrib}/CollegePlaying.csv')
dfbat = pd.read_csv(f'{Core}/Batting.csv')
dfawards = pd.read_csv(f'{Contrib}/AwardsPlayers.csv')

### Analysis 1) From which schools and state did most of the Baseball players are from.

In [ ]:
dfalltable1 = pd.merge(dfschool, dfcollege, on = 'schoolID')

In [ ]:
dfalltable2 = pd.merge(dfalltable1, dfplayers, on = 'playerID')

In [ ]:
df_ds = dfalltable2[['playerID', 'nameFirst', 'nameLast', 'nameGiven', 'yearID', 'schoolID', 'name_full', 'state', 'country']]
df_group = df_ds.groupby(['playerID','yearID']).last().reset_index().sort_values(["playerID", "yearID"], ascending = False)
df_final = df_group.groupby('playerID').nth(0).reset_index() 
df_final1 = df_final

In [ ]:
df_final2 = df_final1.groupby('name_full').agg({'playerID':['count']}).reset_index()
df_final2.columns = ['name_full', 'player_count']
df_final3 = df_final2.sort_values(["player_count"], ascending = False).head(5)

In [ ]:
plt.figure(figsize = (11.5,5))
p = sns.barplot(data = df_final3, x = 'name_full', y = 'player_count')
p.tick_params(labelsize = 8)
plt.title('Top 5 Schools from which most of the players came from', fontsize = 20)
plt.xlabel('School Name', fontsize = 14)
plt.ylabel('Number of Players', fontsize = 14)
plt.show()

In [ ]:
dfcomp = dfplayers.groupby('birthState').agg({'playerID':['count']}).reset_index()
dfcomp.columns = ['statename', 'player_count']
dfcomp1 = dfcomp.sort_values(["player_count"], ascending = False).head(10)

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(data = dfcomp1, x = 'statename', y = 'player_count',)
plt.title('Top 10 States from which most of the players born', fontsize = 20)
plt.xlabel('State Name', fontsize = 14)
plt.ylabel('Number of Players', fontsize = 14)
plt.show()

### Analysis 2) Stolen Base analysis by year

In [ ]:
#Stolen Base
dfsb = dfbat[['yearID', 'SB']]

In [ ]:
dfsbfinal = dfsb.groupby(['yearID']).agg({'SB':['sum']}).reset_index() 
dfsbfinal.columns = ['yearId', 'SB']

In [ ]:
plt.figure(figsize = (10,5))
sns.lineplot(data = dfsbfinal, x = 'yearId', y = 'SB')
plt.title('Stolen Bases by Year', fontsize = 20)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Number of Stolen Base', fontsize = 14)
plt.show()

### Analysis 3) Find the top ten 10 players with respect to their Home Run Rate.

In [ ]:
batting = dfbat
df_hr_rate = batting.groupby(['playerID','yearID']).agg({'AB':'sum','SO':'sum','HR':'sum'}).reset_index()
df_hr_rate = df_hr_rate[df_hr_rate.AB >= 400]
df_hr_rate['rate'] = df_hr_rate['HR'] / (df_hr_rate['AB'] - df_hr_rate['SO'])

In [ ]:
top10_HR_rates = df_hr_rate.sort_values('rate',ascending = False).head(10)
df_people = dfplayers
top10_HR_rates = pd.merge(top10_HR_rates,df_people[['playerID','nameLast']], on = 'playerID')

In [ ]:
plt.figure(figsize = (7,5))
sns.scatterplot(data = top10_HR_rates, x = 'yearID', y ='rate', hue='nameLast')
plt.title('Top 10 Players Home Run Rate by Year', fontsize = 20)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Home Run Rate', fontsize = 14)
plt.show()

### Let's also look at over all home run rate of all the players for all given years.

In [ ]:
df_hr_rate_overall = batting.groupby('yearID').agg({'AB':'sum','SO':'sum','HR':'sum'}).reset_index()

In [ ]:
df_hr_rate_overall['rate'] = df_hr_rate_overall['HR'] / (df_hr_rate_overall['AB'] - df_hr_rate_overall['SO'])

In [ ]:
HR_rates_over = df_hr_rate_overall.sort_values('rate',ascending = False)

In [ ]:
plt.figure(figsize = (8,5))
sns.scatterplot(x = 'yearID', y = 'rate',data = HR_rates_over)
plt.title('Overall Players Home Run Rate by Year', fontsize = 20)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Home Run Rate', fontsize = 14)
plt.show()

### Analysis 4) Change in average salary of all players through out the years:

In [ ]:
salaries_year = dfsal.groupby('yearID').agg({'salary':'mean'}).reset_index()
plt.figure(figsize = (10,5))
sns.lineplot(data = salaries_year, x = 'yearID', y = 'salary')
plt.title('Players Average Salary by Year', fontsize = 20)
plt.xlabel('Year', fontsize = 14)
plt.ylabel('Average Salary', fontsize = 14)
plt.show()

#### Bar plot for average salary of teams for through years:


In [ ]:
salaries = dfsal.groupby('teamID').agg({'salary':'mean'}).reset_index()
plt.figure(figsize = (15,5))
sns.barplot(data = salaries, x = 'teamID', y = 'salary')
plt.title('Teams Average Salary by Year', fontsize = 20)
plt.xlabel('Team ID', fontsize = 14)
plt.ylabel('Average Salary', fontsize = 14)
plt.show()

### Analysis 5) Top 10 players who won highest number of awards

In [ ]:
dfsal.groupby('yearID').agg({'salary':'max', 'playerID':'first'}).reset_index()
dfsal1 = dfsal.sort_values('salary',ascending = False).head(10)

In [ ]:
df_awards = dfawards
df_awards_count = df_awards.groupby('playerID').agg({'awardID':'count'}).reset_index()
df_awards_count.columns = ['playerID','award_count']
top10 = df_awards_count.sort_values('award_count',ascending = False).head(10)

In [ ]:
#merge to get player last names.
top10 = pd.merge(top10,df_people[['playerID','nameLast']], on = 'playerID')

In [ ]:
plt.figure(figsize = (10,5))
sns.barplot(data = top10, x = 'nameLast', y = 'award_count')
plt.title('Top 10 players with most Awards', fontsize = 20)
plt.xlabel('Name', fontsize = 14)
plt.ylabel('Number of Awards', fontsize = 14)
plt.show()